In [1]:
import pandas as pd
import numpy as np

In [107]:
clf_tree.get_params

<bound method Pipeline.get_params of Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='deviance',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
         

In [2]:
df_test= pd.read_csv('test2.csv', header=None)
df_train = pd.read_csv('train2.csv', header=None)

In [3]:
X = df_train.iloc[:,:-1]
y = df_train.iloc[:,-1]

In [4]:
X_test = df_test

In [5]:
#проверим сбалансированность классов
df_train[14].value_counts()
#модель будет предсказывать "0" в 76% случаев, а '1' - в 24% 

0    24720
1     7841
Name: 14, dtype: int64

**Преобразуем тестовые данные**

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
imr = SimpleImputer(missing_values = ' ?', strategy = 'most_frequent')
imr.fit(X)
train_data = imr.transform(X)
train_data = pd.DataFrame(train_data)
train_data[1].value_counts()

 Private             24532
 Self-emp-not-inc     2541
 Local-gov            2093
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: 1, dtype: int64

In [8]:
categorical_features=[1,3,5,6,7,8,9,13]

In [9]:
not_categ = []
for i in range(14): 
    if i not in categorical_features:
        not_categ.append(i)

In [10]:
new_data = pd.DataFrame()
for i in categorical_features: 
    t = pd.get_dummies(train_data[i])
    new_data = pd.concat([new_data, pd.get_dummies(train_data[i])], axis=1)

In [11]:
for i in not_categ:
    pd.concat([new_data, train_data[i]], axis=1, sort=False)

In [ ]:
new_data = new_data.drop(' Holand-Netherlands', axis=1) #этой переменной не будет в тестовой выборке

**Преобразуем тестовые данные**

In [12]:
imr.fit(X_test)
test_data = imr.transform(X_test)
test_data = pd.DataFrame(test_data)
test_data[1].value_counts()

 Private             12173
 Self-emp-not-inc     1321
 Local-gov            1043
 State-gov             683
 Self-emp-inc          579
 Federal-gov           472
 Without-pay             7
 Never-worked            3
Name: 1, dtype: int64

In [13]:
new_test_data = pd.DataFrame()
for i in categorical_features: 
    t = pd.get_dummies(test_data[i])
    new_test_data = pd.concat([new_test_data, pd.get_dummies(test_data[i])], axis=1)
for i in not_categ:
    pd.concat([new_test_data, test_data[i]], axis=1, sort=False)

In [14]:
new_data.shape

(32561, 99)

### **Строим классификацию**

**Метод опорных векторов**

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
#переменных много, сократим размерность с помощью PCA

In [32]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=30)
new = pca.fit(new_data)

print(new.explained_variance_ratio_)
#как видим, нет самых ярких, наиболее значимых, признаков, в основном их совокупность оъясняет 1-2% дисперсии

[0.19653486 0.08035434 0.06831624 0.05874457 0.04849995 0.04765806
 0.04377946 0.03576886 0.03349027 0.02626785 0.02576414 0.02560366
 0.0228597  0.02074752 0.01967692 0.01897551 0.01598888 0.01494046
 0.0130645  0.01191315 0.01104415 0.01090725 0.01014397 0.00980756
 0.00905977 0.00853685 0.00798101 0.00787751 0.00765825 0.0072277 ]


In [47]:
#снизим размерность для ускорения подбора параметров и обучения данных
from sklearn.decomposition import IncrementalPCA
 
ipca = IncrementalPCA(n_components=30)
ipca.fit(new_data)
IncrementalPCA(batch_size=30, n_components=2)
t = ipca.transform(new_data) # doctest: +SKIP

In [48]:
t_df = pd.DataFrame(t)

In [49]:
t_train, t_val, y_train, y_val = train_test_split(t, y, random_state=0)

In [51]:
clf = SVC()
clf.fit(t_train, y_train)
clf.score(t_val, y_val)
#так он обучился быстрее

0.8272939442328953

In [128]:
#переберем параметры RandomizedGridSearch
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [106]:
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_val, y_val)
#так он обучился быстрее

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.8210293575727797

In [73]:
tp = {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
search = RandomizedSearchCV(clf, tp, cv=5, scoring='accuracy')

In [ ]:
#теперь подберем параметры с помощью gridsearchCV
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]},
                   {'kernel': ['poly'], 'degree':[3,6,10,20,100]}]

In [74]:
search.fit(t_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovr',
                                 degree=3, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, probability=False,
                                 random_state=None, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'C': [1, 10, 100, 1000],
                                        'gamma': [0.001, 0.0001],
                                        'kernel': ['rbf']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=0)

In [75]:
search.best_params_

{'kernel': 'rbf', 'gamma': 0.001, 'C': 1000}

In [76]:
search.best_score_ #на бэйзлайне, совсем без подбора, было лучше

0.8226863226863227

In [79]:
clf_svm =make_pipeline(StandardScaler(), SVC(kernel= 'rbf', gamma= 0.001, C=1000)) 

In [81]:
X_train, X_val, y_train, y_val = train_test_split(new_data, y, random_state=0)

In [82]:
clf_svm.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=0.001,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [83]:
clf_svm.score(X_val, y_val)

0.8231175531261515

In [ ]:
#без учета сокращения размерности

In [34]:
X_train, X_val, y_train, y_val = train_test_split(new_data, y, random_state=0)

In [127]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#посмотрим, как будет себя вести базовая версия классификатора
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

In [36]:
clf.score(X_val, y_val) #бэйзлайн для svm

0.8261884289399337

In [ ]:
#Сравним с базовым вариантом решающих деревьев

In [87]:
from sklearn.ensemble import GradientBoostingClassifier

In [88]:
clf_tree = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0))
clf_tree.fit(X_train, y_train)

clf_tree.score(X_val, y_val)#бэйзлайн для GradientBoostingClassifier

0.827171109200344

**промежуточный итог**: \
accuracy на бэйзлайне у бустинга над деревьями: 0.827\
accuracy на бэйзлайне у support vector machine: 0.821

In [ ]:
parameters = {
    "loss":["deviance"],
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "max_depth":[23,25,28],
    "max_features":["log2","sqrt"],
#     "criterion": ["friedman_mse",  "mae"],
#     "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10,15,20,25]
    } 

In [ ]:
search = GridSearchCV(clf, tuned_parameters, cv=5, scoring='accuracy')

In [85]:
#search best_params здесь:
clf_tree_non_scale = GradientBoostingClassifier(random_state=0, max_depth=30, n_estimators=30,learning_rate=0.3)
clf_tree_non_scale.fit(X_train, y_train)

clf_tree_non_scale.score(X_val, y_val) #very 

0.814641935880113

так как search.best_param_ дал результат ниже, чем бэйзлайн, увеличим число ступеней бустинга (n_estimators)

In [110]:
clf_tree = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0,n_estimators=130))
clf_tree.fit(X_train, y_train)

clf_tree.score(X_val, y_val)

0.8282766244933055

In [111]:
clf_tree = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0,n_estimators=150))
clf_tree.fit(X_train, y_train)

clf_tree.score(X_val, y_val)

0.8288907996560619

In [125]:
clf_tree = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0,n_estimators=200))
clf_tree.fit(X_train, y_train)

clf_tree.score(X_val, y_val)
#the best model so far

0.8298734799164722

In [129]:
import csv

In [126]:
preds = clf_tree.predict(new_test_data)
preds = pd.DataFrame(preds)

preds.to_csv('forecast_tree.csv', sep='\t', header=None, index=None)
df = pd.read_csv('forecast_tree.csv')
df.iloc[:,-1].value_counts()

#сохраним результат, оценим разбиение классов - пропорция, совпадает  с тренировочной

0    13334
1     2946
Name: 0, dtype: int64

In [ ]:
clf_tree = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0))
clf_tree.fit(X_train, y_train)

clf_tree.score(X_val, y_val)

**Вывод**

решающие деревья (в том числе с использованием градиентного бустинга) разделяют пространство на гипер-прямоугольники по принципу прироста информации, а метод опорных векторов использует ядерный метод,  наращивая измерения данных до тех пор, пока не найдется линейно разделяющее класссы гиперпространство. \
\
в силу большого объема номинативных переменных, обучение SVM было долгим и не очень эффективным (даже с учетом попыток сокращения размерности с помощью PCA). то же самое можно сказать и о деревьях, но благодаря увеличению стандартного количества итераций бустинга в два раза (до 200), и не меняя при этом остальные параметры, удалось на 0.3% увеличить accuracy деревьев от бэйзлайна 


**промежуточный итог**: \
accuracy на бэйзлайне у бустинга над деревьями: 0.827\
accuracy на бэйзлайне у support vector machine: 0.821

accuracy итоговой модели: 0.83